In [ ]:
import os
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', -1)
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
# Import relevant dataframes:

main_table = pd.read_csv(r"C:\Users\Public\_Data\_Data\Latest_Version\Main_Data\Main_Table_10182019.csv")
print(len(main_table))
faculty_table = pd.read_csv(r"C:\Users\Public\_Data\_Data\Latest_Version\Faculty_Data\Faculty_Table_11222019.csv")
print(len(faculty_table))
skill_table = pd.read_csv(r"C:\Users\Public\_Data\_Data\Latest_Version\Skills_Data\Skill_Table_06072019.csv")
print(len(skill_table))

In [ ]:
def title_swap(string):
    dictionary = {'Community and Social Services':'Counseling and Religious Life',
                  'Customer and Client Support':'Online Support and University Information',
                  'Hospitality, Food, and Tourism':'Event Management and Hospitality', 
                  'Planning and Analysis':'Analysis', 
                  'Curriculum and Instructional Designer / Developer':'Curriculum and Instructional Designer', 
                  'Special Education Teacher':'Accessibility and Disability Services', 
                  'Teaching Assistant':'Faculty Support', 
                  'Tutor':'Academic Tutor',
                  'Clerical and Administrative':'Administrative',
                  'na' : 'Uncategorized'}
    if(string in dictionary):
        return dictionary[string]
    else:
        return string
    
def val_year(year1, year2):
    if(year1 >= year2):
        if(year1>year2):
            temp = year1
            year1 = year2
            year2 = temp
        else:
            raise ValueError('Years cannot be the same.')
            
    return (year1, year2)

def cat_validate(actual, desired_list):
    return actual in desired_list

In [ ]:
post_doc = main_table[main_table['Job Title'].str.contains('postdoc', case=False, na=False) |
                      main_table['Job Title'].str.contains('post doc', case=False, na=False)|
                      main_table['Job Title'].str.contains('post-doc', case=False, na=False)]
print(len(post_doc))

In [ ]:
mask = faculty_table[faculty_table['Post-Doctoral'] == 1][['Job ID']]
print(len(mask))

In [ ]:
boolean_vec = (main_table['Job Title'].str.contains('postdoc', case=False, na=False) |
                      main_table['Job Title'].str.contains('post doc', case=False, na=False)|
                      main_table['Job Title'].str.contains('post-doc', case=False, na=False))

main_table['post_doc'] = boolean_vec + 0

post = faculty_table.merge(main_table[['Job ID', 'post_doc']], on='Job ID', how='inner')

In [ ]:
display(post['Post-Doctoral'].value_counts())

post['Post-Doctoral'] = post['Post-Doctoral'] + post['post_doc']

display(post['Post-Doctoral'].value_counts())

post['Post-Doctoral'] = post['Post-Doctoral'].where(post['Post-Doctoral']<2, 1)

display(post['Post-Doctoral'].value_counts())

In [ ]:
post.drop(columns=['Unnamed: 0', 'post_doc']).to_csv(r"C:\Users\Public\_Data\_Data\Latest_Version\Faculty_Data\Faculty_Table_11222019.csv", index=False)

In [ ]:
def get_categorical_sizes(df, category):
    df_size = df['Job ID'].nunique()
    cat = pd.DataFrame(df[category].value_counts()).reset_index().rename(
        columns={category:'count'})
    cat['inc'] = np.true_divide(cat['count'], df_size)
    return cat

def get_growth(df1, df2, cutoff = 30, min_rate=0.025):
    df1 = df1[df1['inc']>=min_rate]
    df2 = df2[df2['inc']>=min_rate]
    final_df = df1.merge(df2, on='index', how='outer', indicator = True)
    final_df = final_df[final_df['count_y']>=cutoff]
    final_df['growth'] = np.true_divide(final_df['inc_y']-final_df['inc_x'], final_df['inc_x'])
    return final_df

In [ ]:
mask = faculty_table[faculty_table['Post-Doctoral']==1][['Job ID']]
post = main_table.merge(mask, on='Job ID', how='inner')

total = post.shape[0]

post = post[~post['IPEDS Institution Name'].isnull()]

print("Number of missing schools: " + str(total-post.shape[0]))

year1 = 2010
year2 = 2017

post1 = get_categorical_sizes(post[post['Year']==year1], 'IPEDS Institution Name')
post2 = get_categorical_sizes(post[post['Year']==year2], 'IPEDS Institution Name')

growth = get_growth(post1, post2, cutoff=30, min_rate=0.001)

growth.sort_values(by='growth', ascending=False)

In [ ]:
zone1 = get_categorical_sizes(post[post['Year']==year1], 'BEA_Zone')
zone2 = get_categorical_sizes(post[post['Year']==year2], 'BEA_Zone')

growth = get_growth(zone1, zone2, cutoff=30, min_rate=0.001)

growth.sort_values(by='growth', ascending=False)

In [ ]:
post_fac = faculty_table.merge(mask)

In [ ]:
print(len(skill_table))

In [ ]:
skill_table['Job ID'].nunique()